In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DMG-intern/kaggle/fakenews.csv')[:100]
print(train_df)

              id                                            context  isfake  \
0   000128042337  朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コン...       0   
1   00012b7a8314  11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置...       2   
2   0005fb48880b  産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2...       2   
3   00087f9e14ab  開催地のリオデジャネイロ市に対して、大会期間中のリオデジャネイロオリンピックに関する公式発表...       1   
4   000c9ac3d552  毎日新聞・時事通信によると、2006年2月13日には、グッドウィル・グッゲンハイム・アン・ハ...       2   
..           ...                                                ...     ...   
95  01d8e63d3083  第88回全国高校野球選手権大会の南北海道地区大会決勝が25日に札幌円山球場で行われ、昨年2連...       0   
96  01e0aec286ec  北陸中日新聞によると、地震による津波は想定以上に大きく、この影響で気象庁は13日11時36分...       2   
97  01e11084f4b0  朝日新聞・読売新聞によると、2009年6月（UTC+9）に兵庫県警佐用署に軽犯罪法違反容疑で...       0   
98  01f104ed8276  毎日新聞、朝日新聞によると、テロ組織の名前である「」が、公式ウェブサイト上で「AISM」と名...       2   
99  01fcd1044e3f  2000年より日本に滞在しているペルー元大統領アルベルト・フジモリ氏は、6日東京で行った記者...       0   

    nchar_real  nchar_fake  
0          541        

In [ ]:
print(train_df.context)

0     朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コン...
1     11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置...
2     産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2...
3     開催地のリオデジャネイロ市に対して、大会期間中のリオデジャネイロオリンピックに関する公式発表...
4     毎日新聞・時事通信によると、2006年2月13日には、グッドウィル・グッゲンハイム・アン・ハ...
                            ...                        
95    第88回全国高校野球選手権大会の南北海道地区大会決勝が25日に札幌円山球場で行われ、昨年2連...
96    北陸中日新聞によると、地震による津波は想定以上に大きく、この影響で気象庁は13日11時36分...
97    朝日新聞・読売新聞によると、2009年6月（UTC+9）に兵庫県警佐用署に軽犯罪法違反容疑で...
98    毎日新聞、朝日新聞によると、テロ組織の名前である「」が、公式ウェブサイト上で「AISM」と名...
99    2000年より日本に滞在しているペルー元大統領アルベルト・フジモリ氏は、6日東京で行った記者...
Name: context, Length: 100, dtype: object


## 前処理
参考：https://note.com/narudesu/n/na35de30a583a#xIxeK

### クリーニング

In [ ]:
!pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neologdn: filename=neologdn-0.5.1-cp38-cp38-linux_x86_64.whl size=208755 sha256=9ac3d0f1d149f381e7d88a4703a10fa158ad566de3f1f40f0dcefc494f938504
  Stored in directory: /root/.cache/pip/wheels/3e/db/10/b3b26caa63c5da86ea3a25043cc4379a66bb3dd30d6f060a37
Successfully built neologdn


In [ ]:
import neologdn
import re

for text in train_df.context:
  # 改行コードの削除
  text = text.replace('\n','').replace('\r','')
  # URL削除
  text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
  text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
  # 記号削除を半角スペースに置換
  text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
  text = text_removed_symbol = re.sub(u'[■-♯]', ' ', text)
  # 全角英数字を半角に変換・1回以上連続する長音記号を1回に変換（その他正規化処理）
  text = neologdn.normalize(text)
  # 桁区切り文字を0に変換に変換
  text = re.sub(r'\b\d{1,3}(,\d{3})*\b', '0', text)
  # 数値を全て 0 に変換
  text = re.sub(r'\d+', '0', text)
  # 大文字を小文字に統一
  text = text.lower()

### 単語分割

In [59]:
# install MeCab
!sudo apt update
# !apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
# !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
# !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!sudo apt install mecab
# !pip install mecab-python3

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [56]:
# !ln -s /usr/local/lib/python3.8/dist-packages/mecab-config /usr/local/bin/mecab-config

ln: failed to create symbolic link '/usr/local/bin/mecab-config': File exists


In [48]:
# !echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/bin/bash: mecab-config: command not found
/mecab-ipadic-neologd


In [60]:
import MeCab

for text in train_df.context:
  tagger = Mecab.Tagger()
  for line in tagger.parse(text).splitlines():
    print(line)

NameError: ignored

## モデルの構築と予測

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
	train_df['context'].values.astype('str'),
	train_df['isfake'].values.astype('int'),
	test_size=0.2,
	random_state=0
)

In [ ]:
# コーパスクリーニング

print(train_df['context'])